In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.datasets import mnist
from sklearn.utils import shuffle
from skimage.transform import resize

In [2]:
# Load MNIST dataset
(X, y), (X_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
#Reduce size to make it a "reduced MNIST"
# Resize from 28x28 to 14x14
X_resized = np.array([resize(img, (14, 14), anti_aliasing=True) for img in X])
X_test_resized = np.array([resize(img, (14, 14), anti_aliasing=True) for img in X_test])

In [4]:
# Flatten the images
X_flat = X_resized.reshape(X_resized.shape[0], -1)
X_test_flat = X_test_resized.reshape(X_test_resized.shape[0], -1)

In [5]:
# Use only a subset of data (reduce to 10000 samples)
X_flat, y = shuffle(X_flat, y, random_state=42)
X_subset = X_flat[:10000]
y_subset = y[:10000]

In [6]:
# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split(X_subset, y_subset, test_size=0.2, random_state=42)

In [7]:
# Polynomial Features
degree = 2
pR = PolynomialFeatures(degree=degree, include_bias=False)
X_train_pR = pR.fit_transform(X_train)
X_valid_pR = pR.transform(X_valid)

In [8]:
# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_pR)
X_valid_scaled = scaler.transform(X_valid_pR)

In [ ]:
# Train Lasso model
model = Lasso(alpha=0.001, max_iter=10000)
model.fit(X_train_scaled, y_train)

In [ ]:
# Round and clip to valid classes
y_pred_class = np.clip(np.round(y_pred_float), 0, 9).astype(int)

In [ ]:
# Evaluation
accuracy = accuracy_score(y_valid, y_pred_class)
print(f"🎯 Accuracy: {accuracy * 100:.2f}%\n")
print("📄 Classification Report:")
print(classification_report(y_valid, y_pred_class))
print("📊 Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred_class))

In [ ]:
# Visualization
def visualize_predictions(X, y_true, y_pred, n=10):
    plt.figure(figsize=(12, 4))
    for i in range(n):
        plt.subplot(1, n, i + 1)
        plt.imshow(X[i], cmap='gray')
        plt.title(f"T:{y_true[i]}\nP:{y_pred[i]}", color='green' if y_true[i] == y_pred[i] else 'red')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_predictions(X_valid.reshape(-1, 14, 14), y_valid, y_pred_class)